In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
import functools
import numpy as np
import pandas as pd

from scipy.special import expit

import sklearn as sk
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow import keras
import tensorflow as tf

def split_companies_train_dev_test(companies):
    "Return train, dev, test set for companies"
    train, test = train_test_split(companies, test_size=0.1, stratify = companies.sector)
    train, dev = train_test_split(train, test_size=0.1, stratify = train.sector)
    return train, dev, test


def filter_stocks(stocks, tickers):
    return stocks.loc[tickers]


def df_to_ts(df):
    res = df.copy()
    res.index = pd.DatetimeIndex(pd.to_datetime(res.date))
    res.drop('date', axis=1)
    return res


def log_softmax(x):
    return x - np.log(np.sum(np.exp(x)))


def sigmoid(x):
    return expit(x)


def sample_correlation(df, window_size=63):
    idx = np.random.randint(0, df.shape[0]-window_size)
    ts = df[idx:idx+window_size]
    fmap = lambda s: ts['pct_return'].corr(ts[s])
    indices = ts.columns.tolist()[1:]
    correlations = np.array(list(map(fmap, indices)))
    return correlations


def create_correlation_score(df, sample_size=1):
    res = np.array([log_softmax(sample_correlation(df)/0.05)
                    for i in range(sample_size)])
    return np.exp(np.nanmean(res, 0))


def load_data(stock_filename=None, indices_filename=None):

    if stock_filename is None:
        stock_filename = '../../data/processed/wiki_stocks_returns.csv'

    if indices_filename is None:
        indices_filename = '../../data/processed/wiki_indices_returns.csv'

    stocks = pd.read_csv(stock_filename, index_col=False) # long format
    indices = pd.read_csv(indices_filename, index_col=False) # wide format

    # Implementation of hierarchical clustering
    drop_column = lambda df,i=0: df.drop(df.columns[i], axis=1)

    stocks = drop_column(stocks)
    stocks = stocks.drop('name', axis=1)
    stocks = stocks.dropna()

    companies = stocks.groupby('ticker').first().reset_index()
    sectors_counts = companies.sector.value_counts()
    sectors_proportions = sectors_counts/sectors_counts.sum()
    sectors_unique = sectors_counts.index.tolist()

    stocks = stocks.set_index('ticker')

    indices_ts = df_to_ts(indices[['date'] + sectors_unique])
    stocks_ts = df_to_ts(stocks.reset_index())

    stocks_all = pd.merge(stocks_ts, indices_ts, 'left')
    stocks_all = stocks_all.dropna() # loss of 200 000 observations
    stocks_all = stocks_all.drop('sector', axis=1)
    stocks_all = stocks_all.groupby('ticker').apply(df_to_ts)
    stocks_all = stocks_all.drop(['ticker', 'date'], axis=1)
    stocks_all = stocks_all.rename(columns={'close': 'pct_return'})

    label_encoder = preprocessing.LabelEncoder()
    label_encoder.fit(sectors_counts.index.tolist())
    ticker_to_sector = dict(zip(companies.ticker, label_encoder.transform(companies.sector)))

    return stocks_all, companies, label_encoder, ticker_to_sector

def sectors_statistics(companies):
    sectors_counts = companies.sector.value_counts()
    sectors_proportions = sectors_counts/sectors_counts.sum()
    sectors_unique = sectors_counts.index.tolist()
    return sectors_counts, sectors_proportions, sectors_unique


def add_common_layers(y):
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.LeakyReLU()(y)
    return y


def grouped_convolution(y, nb_channels, _strides, cardinality=4):
    # when `cardinality` == 1 this is just a standard convolution
    if cardinality == 1:
        return keras.layers.Conv1D(nb_channels, kernel_size=10, strides=_strides, padding='same')(y)

    assert not nb_channels % cardinality
    _d = nb_channels // cardinality

    # in a grouped convolution layer, input and output channels are divided into `cardinality` groups,
    # and convolutions are separately performed within each group
    groups = []
    for j in range(cardinality):
        group = keras.layers.Lambda(lambda z: z[:, :, j * _d:j * _d + _d])(y)
        groups.append(keras.layers.Conv1D(_d, kernel_size=10, strides=_strides, padding='same')(group))

    # the grouped convolutional layer concatenates them as the outputs of the layer
    y = keras.layers.concatenate(groups)

    return y


def residual_block(y, nb_channels_in, nb_channels_out, cardinality=4, _strides=1, _project_shortcut=False):
    """
    Our network consists of a stack of residual blocks. These blocks have the same topology,
    and are subject to two simple rules:
    - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
    - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
    """
    shortcut = y
    kl = keras.layers
    # we modify the residual building block as a bottleneck design to make the network more economical
    y = kl.Conv1D(nb_channels_in, kernel_size=1, strides=1, padding='same')(y)
    y = add_common_layers(y)

    # ResNeXt (identical to ResNet when `cardinality` == 1)
    y = grouped_convolution(y, nb_channels_in, _strides=_strides)
    y = add_common_layers(y)

    y = kl.Conv1D(nb_channels_out, kernel_size=1, strides=1, padding='same')(y)
    # batch normalization is employed after aggregating the transformations and before adding to the shortcut
    y = kl.BatchNormalization()(y)

    # identity shortcuts used directly when the input and output are of the same dimensions
    if _project_shortcut or _strides != 1:
        # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
        # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
        shortcut = kl.Conv1D(nb_channels_out, kernel_size=1, strides=_strides, padding='same')(shortcut)
        shortcut = kl.BatchNormalization()(shortcut)

    y = kl.add([shortcut, y])

    # relu is performed right after each batch normalization,
    # expect for the output of the block where relu is performed after the adding to the shortcut
    y = kl.LeakyReLU()(y)

    return y



# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = keras.backend.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = tf.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class CovarianceLayer(keras.layers.Layer):

    def __init__(self, num_classes=None, **kwargs):
        if num_classes is None:
            num_classes = 16
        self.num_classes = num_classes
        super(CovarianceLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        super(CovarianceLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        series_input, environment_input = inputs
        series_input_multiple = tf.tile(series_input, [1, 1, self.num_classes])
        covariances = tf.reduce_mean(series_input_multiple * environment_input, axis=1);
        return covariances

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


def random_subset(df, window_size=21):
    idx = np.random.randint(0, df.shape[0]-window_size)
    ts = df[idx:idx+window_size]
    return ts


def make_keras_subset(dataset_type, companies_data, stocks_data, label_encoder, batch_size, window_size=21):
    idx = np.random.choice(companies_data[dataset_type].shape[0], batch_size)
    df = companies_data[dataset_type].iloc[idx]

    model_input_data = [random_subset(stocks_data[dataset_type].loc[t], window_size) for t in df.ticker]
    model_series_input = np.array([df['pct_return'].values for df in model_input_data])
    model_series_input = model_series_input.reshape(-1, window_size, 1)

    model_environment_input = np.array([df.iloc[:, 1:].values for df in model_input_data])

    y_true = label_encoder.transform(df.sector)

    return model_series_input, model_environment_input, y_true


class StocksSequence(keras.utils.Sequence):

    def __init__(self, stocks_data,  companies_data, window_size, label_encoder, batch_size):
        self.stocks_data = stocks_data
        self.batch_size = batch_size
        self.label_encoder = label_encoder
        self.companies_data = companies_data
        self.window_size = window_size

    def __len__(self):
        return int(np.ceil(self.stocks_data.shape[0] / float(self.batch_size)))

    def __getitem__(self, idx):

        idx = np.random.choice(self.companies_data.shape[0], self.batch_size)
        df = self.companies_data.iloc[idx]
        model_input_data = [random_subset(self.stocks_data.loc[t], self.window_size) for t in df.ticker]
        model_series_input = np.array([df['pct_return'].values for df in model_input_data])
        model_series_input = model_series_input.reshape(-1, self.window_size, 1)
        model_environment_input = np.array([df.iloc[:, 1:].values for df in model_input_data])
        y_true = self.label_encoder.transform(df.sector)

        return [model_series_input, model_environment_input], y_true





In [3]:

def make_model(num_classes=16, window_size=21, latent_dim=32):
    kl = keras.layers

    series_input = keras.layers.Input(shape=(window_size, 1), dtype='float32', name='series_input')
    environment_input = kl.Input(shape=(window_size, num_classes), dtype='float32', name='environment_input')
    tanh_layer = kl.Lambda(lambda y: tf.tanh(tf.scalar_mul(5, y)))
    
    x = kl.Concatenate()([series_input, environment_input])
    x = tanh_layer(x)
    x = kl.Conv1D(16, 5, 1, activation='relu')(x)
    x = residual_block(x, 16, 16, 2)
    # x = kl.Dropout(0.001)(x)
    x = kl.Conv1D(32, 5, 2, activation='relu')(x)
    # x = kl.Dropout(0.05)(x)
    x = residual_block(x, 32, 32, 2)
    # x = kl.Dropout(0.05)(x)
    x = kl.Conv1D(64, 5, 2, activation='relu')(x)
    # x = kl.Dropout(0.1)(x)
    # x = kl.MaxPool1D()(x)

    covariances = CovarianceLayer(num_classes)([tanh_layer(series_input), tanh_layer(environment_input)])
    covariances = kl.Dense(32, activation='relu')(covariances)
    covariances = kl.Dense(64, activation='relu')(covariances)

    x = kl.Flatten()(x)
    x = kl.Dense(64, 'relu')(x)
    x = kl.Dropout(0.1)(x)
    z = kl.Add(name='Embedding')([x, covariances])
    z = kl.Dense(64, 'relu')(z)
    z = kl.Dense(32, 'relu')(z)
    x_pred = kl.Dense(num_classes, 'softmax')(z)

    model = keras.Model(inputs = [series_input, environment_input], outputs=[x_pred], name='Classifier')

    # kl_batch = - .5 * tf.reduce_sum(1 + x_log_var - tf.square(x_mu) - tf.exp(x_log_var), axis=-1)
    # model.add_loss(kl_batch)

    return model

In [4]:
# Make train dev test set.
np.random.seed(42)

### Feature engineering

stock_filename = '../data/processed/wiki_stocks_returns.csv'
indices_filename = '../data/processed/wiki_indices_returns.csv'

stocks_all, companies, label_encoder, ticker_to_sector = load_data(stock_filename, indices_filename)
sectors_counts, sectors_proportions, sectors_unique = sectors_statistics(companies)

max_proportion_baseline = sectors_proportions.max()
biggest_sector = sectors_proportions.argmax()

print("Most representated class:", biggest_sector, ', with proportion of ', round(100*max_proportion_baseline, 2), '%.')
# Accuracy of our models should be better than max_proportion_baseline.

companies_data = {}
data_split = split_companies_train_dev_test(companies)
for i, k in enumerate(['train', 'dev', 'test']):
    companies_data[k] = data_split[i]
stocks_data = {k: filter_stocks(stocks_all, v.ticker) for k, v in companies_data.items()}


Most representated class: Financial Services , with proportion of  13.09 %.


In [5]:
window_size = 63
model = make_model(window_size=window_size)

In [6]:
if False:
    model.load_weights('checkpoint/model_weights.json')

In [10]:
import math
def schedule_fn(epoch):
   initial_lrate = 0.001
   drop = 0.8
   epochs_drop = 2.0
   epoch = epoch % 10
   epoch_pow = math.floor((1+epoch)/epochs_drop)
   lrate = initial_lrate * math.pow(drop, epoch_pow)
   return lrate

In [11]:
batch_size = 64

optimizer = keras.optimizers.Adam(0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

callbacks = [
    keras.callbacks.ModelCheckpoint('checkpoint/model_weights.json', monitor='val_acc', verbose=1, save_best_only=True, mode='max'),
    keras.callbacks.TensorBoard('logs/seventh'),
    keras.callbacks.LearningRateScheduler(schedule_fn)
]

stocks_sequence_training = StocksSequence(stocks_data['train'], companies_data['train'], window_size, label_encoder, batch_size)
stocks_sequence_validation = StocksSequence(stocks_data['dev'], companies_data['dev'], window_size, label_encoder, batch_size)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
series_input (InputLayer)       (None, 63, 1)        0                                            
__________________________________________________________________________________________________
environment_input (InputLayer)  (None, 63, 16)       0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 63, 17)       0           series_input[0][0]               
                                                                 environment_input[0][0]          
__________________________________________________________________________________________________
lambda_1 (Lambda)               multiple             0           concatenate_1[0][0]              
          

In [12]:
model.fit_generator(
    stocks_sequence_training, steps_per_epoch=2000, epochs=400, callbacks=callbacks,
    validation_data = stocks_sequence_validation, validation_steps=100, workers=4, max_queue_size=20, verbose=1,
    use_multiprocessing=True)

stocks_sequence_test = StocksSequence(stocks_data['test'], companies_data['test'], window_size, label_encoder, batch_size)

model.evaluate_generator(stocks_sequence_test, 400, workers=2, use_multiprocessing=True)

Epoch 1/400
2000/2000 [==============================]2000/2000 [==============================] - 177s 89ms/step - loss: 1.4125 - acc: 0.5512 - val_loss: 1.8749 - val_acc: 0.4281

Epoch 2/400
2000/2000 [==============================]2000/2000 [==============================] - 159s 80ms/step - loss: 1.4135 - acc: 0.5508 - val_loss: 1.9015 - val_acc: 0.4294

Epoch 3/400
2000/2000 [==============================]2000/2000 [==============================] - 162s 81ms/step - loss: 1.4262 - acc: 0.5408 - val_loss: 1.9054 - val_acc: 0.4272

Epoch 4/400
2000/2000 [==============================]2000/2000 [==============================] - 165s 82ms/step - loss: 1.4311 - acc: 0.5419 - val_loss: 2.0061 - val_acc: 0.4394

Epoch 5/400
2000/2000 [==============================]2000/2000 [==============================] - 161s 80ms/step - loss: 1.4236 - acc: 0.5454 - val_loss: 1.7891 - val_acc: 0.4575

Epoch 6/400
2000/2000 [==============================]2000/2000 [==============================


Epoch 24/400
2000/2000 [==============================]2000/2000 [==============================] - 221s 110ms/step - loss: 1.3819 - acc: 0.5567 - val_loss: 1.6965 - val_acc: 0.4870

Epoch 25/400
2000/2000 [==============================]2000/2000 [==============================] - 224s 112ms/step - loss: 1.3903 - acc: 0.5553 - val_loss: 1.7196 - val_acc: 0.4688

Epoch 26/400
2000/2000 [==============================]2000/2000 [==============================] - 226s 113ms/step - loss: 1.3895 - acc: 0.5557 - val_loss: 1.8123 - val_acc: 0.4569

Epoch 27/400
2000/2000 [==============================]2000/2000 [==============================] - 227s 114ms/step - loss: 1.3866 - acc: 0.5614 - val_loss: 1.7435 - val_acc: 0.4763

Epoch 28/400
2000/2000 [==============================]2000/2000 [==============================] - 230s 115ms/step - loss: 1.3827 - acc: 0.5599 - val_loss: 1.8198 - val_acc: 0.4512

Epoch 29/400
2000/2000 [==============================]2000/2000 [==================


Epoch 48/400
2000/2000 [==============================]2000/2000 [==============================] - 227s 114ms/step - loss: 1.3566 - acc: 0.5671 - val_loss: 1.6998 - val_acc: 0.4853

Epoch 49/400
2000/2000 [==============================]2000/2000 [==============================] - 225s 113ms/step - loss: 1.3164 - acc: 0.5788 - val_loss: 1.7003 - val_acc: 0.4806

Epoch 50/400
2000/2000 [==============================]2000/2000 [==============================] - 223s 111ms/step - loss: 1.3216 - acc: 0.5757 - val_loss: 1.6510 - val_acc: 0.4931

Epoch 51/400
2000/2000 [==============================]2000/2000 [==============================] - 224s 112ms/step - loss: 1.3689 - acc: 0.5619 - val_loss: 1.8176 - val_acc: 0.4525

Epoch 52/400
2000/2000 [==============================]2000/2000 [==============================] - 223s 112ms/step - loss: 1.3743 - acc: 0.5627 - val_loss: 1.7898 - val_acc: 0.4516

Epoch 53/400
2000/2000 [==============================]2000/2000 [==================


Epoch 72/400
2000/2000 [==============================]2000/2000 [==============================] - 221s 111ms/step - loss: 1.3268 - acc: 0.5784 - val_loss: 1.6985 - val_acc: 0.4747

Epoch 73/400
2000/2000 [==============================]2000/2000 [==============================] - 224s 112ms/step - loss: 1.3097 - acc: 0.5838 - val_loss: 1.7392 - val_acc: 0.4703

Epoch 74/400
2000/2000 [==============================]2000/2000 [==============================] - 222s 111ms/step - loss: 1.3205 - acc: 0.5811 - val_loss: 1.6575 - val_acc: 0.5094

Epoch 75/400
2000/2000 [==============================]2000/2000 [==============================] - 225s 113ms/step - loss: 1.2964 - acc: 0.5878 - val_loss: 1.6993 - val_acc: 0.4831

Epoch 76/400
2000/2000 [==============================]2000/2000 [==============================] - 226s 113ms/step - loss: 1.2874 - acc: 0.5897 - val_loss: 1.6702 - val_acc: 0.4819

Epoch 77/400
2000/2000 [==============================]2000/2000 [==================


Epoch 96/400
2000/2000 [==============================]2000/2000 [==============================] - 228s 114ms/step - loss: 1.2987 - acc: 0.5890 - val_loss: 1.6704 - val_acc: 0.4761

Epoch 97/400
2000/2000 [==============================]2000/2000 [==============================] - 226s 113ms/step - loss: 1.2851 - acc: 0.5935 - val_loss: 1.6523 - val_acc: 0.5006

Epoch 98/400
2000/2000 [==============================]2000/2000 [==============================] - 224s 112ms/step - loss: 1.2753 - acc: 0.5932 - val_loss: 1.6367 - val_acc: 0.5081

Epoch 99/400
2000/2000 [==============================]2000/2000 [==============================] - 229s 114ms/step - loss: 1.2793 - acc: 0.5912 - val_loss: 1.6476 - val_acc: 0.4908

Epoch 100/400
2000/2000 [==============================]2000/2000 [==============================] - 229s 114ms/step - loss: 1.2734 - acc: 0.5963 - val_loss: 1.6160 - val_acc: 0.5094

Epoch 101/400
2000/2000 [==============================]2000/2000 [================

...


Epoch 312/400
2000/2000 [==============================]2000/2000 [==============================] - 144s 72ms/step - loss: 1.1611 - acc: 0.6344 - val_loss: 1.6262 - val_acc: 0.5134

Epoch 313/400
2000/2000 [==============================]2000/2000 [==============================] - 148s 74ms/step - loss: 1.1760 - acc: 0.6286 - val_loss: 1.6700 - val_acc: 0.4961

Epoch 314/400
2000/2000 [==============================]2000/2000 [==============================] - 146s 73ms/step - loss: 1.1766 - acc: 0.6289 - val_loss: 1.6633 - val_acc: 0.5131

Epoch 315/400
2000/2000 [==============================]2000/2000 [==============================] - 147s 73ms/step - loss: 1.1420 - acc: 0.6380 - val_loss: 1.6288 - val_acc: 0.5294

Epoch 316/400
2000/2000 [==============================]2000/2000 [==============================] - 146s 73ms/step - loss: 1.1719 - acc: 0.6314 - val_loss: 1.5694 - val_acc: 0.5275

Epoch 317/400
2000/2000 [==============================]2000/2000 [=================


Epoch 336/400
2000/2000 [==============================]2000/2000 [==============================] - 146s 73ms/step - loss: 1.1856 - acc: 0.6271 - val_loss: 1.6306 - val_acc: 0.5128

Epoch 337/400
2000/2000 [==============================]2000/2000 [==============================] - 143s 71ms/step - loss: 1.1346 - acc: 0.6428 - val_loss: 1.6450 - val_acc: 0.5102

Epoch 338/400
2000/2000 [==============================]2000/2000 [==============================] - 144s 72ms/step - loss: 1.1514 - acc: 0.6367 - val_loss: 1.5796 - val_acc: 0.5311

Epoch 339/400
2000/2000 [==============================]2000/2000 [==============================] - 145s 72ms/step - loss: 1.1686 - acc: 0.6303 - val_loss: 1.5915 - val_acc: 0.5294

Epoch 340/400
2000/2000 [==============================]2000/2000 [==============================] - 144s 72ms/step - loss: 1.1540 - acc: 0.6351 - val_loss: 1.5637 - val_acc: 0.5337

Epoch 341/400
2000/2000 [==============================]2000/2000 [=================


Epoch 360/400
2000/2000 [==============================]2000/2000 [==============================] - 144s 72ms/step - loss: 1.1301 - acc: 0.6408 - val_loss: 1.6075 - val_acc: 0.5062

Epoch 361/400
2000/2000 [==============================]2000/2000 [==============================] - 144s 72ms/step - loss: 1.1801 - acc: 0.6261 - val_loss: 1.8201 - val_acc: 0.4780

Epoch 362/400
2000/2000 [==============================]2000/2000 [==============================] - 142s 71ms/step - loss: 1.1648 - acc: 0.6345 - val_loss: 1.8483 - val_acc: 0.4878

Epoch 363/400
2000/2000 [==============================]2000/2000 [==============================] - 142s 71ms/step - loss: 1.1534 - acc: 0.6353 - val_loss: 1.6424 - val_acc: 0.5191

Epoch 364/400
2000/2000 [==============================]2000/2000 [==============================] - 143s 71ms/step - loss: 1.1516 - acc: 0.6361 - val_loss: 1.6421 - val_acc: 0.5064

Epoch 365/400
2000/2000 [==============================]2000/2000 [=================


Epoch 384/400
2000/2000 [==============================]2000/2000 [==============================] - 140s 70ms/step - loss: 1.1333 - acc: 0.6401 - val_loss: 1.6206 - val_acc: 0.5253

Epoch 385/400
2000/2000 [==============================]2000/2000 [==============================] - 140s 70ms/step - loss: 1.1340 - acc: 0.6391 - val_loss: 1.6681 - val_acc: 0.5211

Epoch 386/400
2000/2000 [==============================]2000/2000 [==============================] - 142s 71ms/step - loss: 1.1387 - acc: 0.6384 - val_loss: 1.6077 - val_acc: 0.5256

Epoch 387/400
2000/2000 [==============================]2000/2000 [==============================] - 143s 72ms/step - loss: 1.1676 - acc: 0.6323 - val_loss: 1.6173 - val_acc: 0.5188

Epoch 388/400
2000/2000 [==============================]2000/2000 [==============================] - 142s 71ms/step - loss: 1.1376 - acc: 0.6423 - val_loss: 1.6281 - val_acc: 0.5339

Epoch 389/400
2000/2000 [==============================]2000/2000 [=================

[1.521976063847542, 0.5409375]